In [15]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
from haikunator import Haikunator
import os.path as path

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print(x_train.shape, x_test.shape)
print(np.min(x_train), np.max(x_train))
print(np.min(x_test), np.max(x_test))

(60000, 28, 28, 1) (10000, 28, 28, 1)
0.0 1.0
0.0 1.0


In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [8]:
class MyModel(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(10, activation="softmax")

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [9]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optim = tf.keras.optimizers.Adam()

In [10]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [17]:
@tf.function
def train_step(model, images, labels):
    with tf.GradientTape() as tape:
        preds = model(images, training=True)
        loss = loss_fn(labels, preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_acc(labels, preds)

In [18]:
@tf.function
def test_step(model, images, labels):
    preds = model(images, training=False)
    loss = loss_fn(labels, preds)
    test_loss(loss)
    test_acc(labels, preds)


In [19]:
runid = Haikunator().haikunate()
train_logdir = path.expanduser(f"~/mldata/tblogs/mnist/{runid}/train")
test_logdir = path.expanduser(f"~/mldata/tblogs/mnist/{runid}/test")
train_sw = tf.summary.create_file_writer(train_logdir)
test_sw = tf.summary.create_file_writer(test_logdir)
print(train_logdir)
print(test_logdir)

/Users/avilay/mldata/tblogs/mnist/spring-hat-9222/train
/Users/avilay/mldata/tblogs/mnist/spring-hat-9222/test


In [20]:
EPOCHS = 5
model = MyModel()

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()

    for images, labels in train_ds:
        train_step(model, images, labels)

    with train_sw.as_default():
        tf.summary.scalar("loss", train_loss.result(), step=epoch)
        tf.summary.scalar("accuracy", train_acc.result()*100, step=epoch)

    for images, labels in test_ds:
        test_step(model, images, labels)

    with test_sw.as_default():
        tf.summary.scalar("loss", test_loss.result(), step=epoch)
        tf.summary.scalar("accuracy", train_acc.result()*100, step=epoch)

    tmpl = "Epoch {}, Loss: {:.3f}, Accuracy: {:.3f}, Test Loss: {:.3f}, Test Accuracy: {:.3f}"
    print(tmpl.format(epoch+1, train_loss.result(), train_acc.result()*100, test_loss.result(), test_acc.result() * 100))

Epoch 1, Loss: 0.108, Accuracy: 96.747, Test Loss: 0.053, Test Accuracy: 98.250
Epoch 2, Loss: 0.027, Accuracy: 99.170, Test Loss: 0.055, Test Accuracy: 98.260
Epoch 3, Loss: 0.013, Accuracy: 99.597, Test Loss: 0.047, Test Accuracy: 98.540
Epoch 4, Loss: 0.008, Accuracy: 99.762, Test Loss: 0.053, Test Accuracy: 98.540
Epoch 5, Loss: 0.005, Accuracy: 99.822, Test Loss: 0.064, Test Accuracy: 98.370
